# Fire Spotter
(Credit to jhoward and his fast.ai class for teaching me this code and providing this library!)

Have you ever wanted an AI to confirm for you that your apartment is, in fact, on fire? Well, I've got just the tool!

🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥

In [1]:
#Imports and environment crap
import socket, warnings, os, time, json
from duckduckgo_search import DDGS
from fastcore.all import *
from fastdownload import download_url
from fastai.vision.all import *

try:
    socket.setdefaulttimeout(1)
    socket.socket(socket.AF_INET, socket.SOCK_STREAM).connect(('1.1.1.1', 53))
except socket.error as ex: raise Exception("STOP: No internet. Click '>|' in top right and set 'Internet' switch to on")

iskaggle = os.environ.get('KAGGLE_KERNEL_RUN_TYPE', '')

if iskaggle:
    !pip install -Uqq fastai 'duckduckgo_search>=6.2'

ModuleNotFoundError: No module named 'duckduckgo_search'

In [ ]:
# func definitions
def search_images(keywords, max_images=200): return L(DDGS().images(keywords, max_results=max_images)).itemgot('image')

## Inital Downloads / Dataset Aquisition
Lets grab some images, confirm everything is all sane, and get squared away to commence into training.

In [ ]:
#Do a sanity check download of an apartment fire
urls = search_images('apartment fire', max_images=1)
urls[0]
dest = 'fire.jpg'
download_url(urls[0], dest, show_progress=False)
im = Image.open(dest)
#Display one to confirm what we're grabbing visually
im.to_thumb(256,256)

In [ ]:
searches = 'apartment fire','apartment'
path = Path('on_fire_or_not')

for o in searches:
    dest = (path/o)
    dest.mkdir(exist_ok=True, parents=True)
    download_images(dest, urls=search_images(f'{o} photo'))
    time.sleep(5)
    resize_images(path/o, max_size=400, dest=path/o)

In [ ]:
# Validation before we continue
# once again credit to jhoward for teaching the importance of validation
failed = verify_images(get_image_files(path))
failed.map(Path.unlink)
len(failed)

## Training
Time to use all of the above to train

In [ ]:
# Populate DataBlocks for PyTorch
dls = DataBlock(
    blocks=(ImageBlock, CategoryBlock), 
    get_items=get_image_files, 
    splitter=RandomSplitter(valid_pct=0.2, seed=42),
    get_y=parent_label,
    item_tfms=[Resize(192, method='squish')]
).dataloaders(path, bs=32)

# Can never have enough sanity checks
dls.show_batch(max_n=6)

In [ ]:
learn = vision_learner(dls, resnet18, metrics=error_rate)
learn.fine_tune(3)

# Results
Are things on fire?

In [ ]:
is_on_fire,_,probs = learn.predict(PILImage.create('fire.jpg'))
print(f"Result: {is_on_fire}.")
print(f"Probability I'm right: {probs[0]:.4f}")
im = Image.open('fire.jpg')
im.to_thumb(256,256)